# PyLASCA
a tool for ...

## Complete Area Alignment
Analyse and compare areas of LaSCA and mechanical pain detection

by

Sebastian Malkusch

2020-06-16

Institute for clinical pharmacology,

data science division,

Goethe-University Frankfurt,

Germany

malkusch@med.uni-frankfurt.de

www.smb.uni-frankfurt.de

http://share.smb.uni-frankfurt.de

## Import libraries

In [1]:
from ipywidgets import IntProgress
from IPython.display import display
import cv2 as cv
import pandas as pd
import numpy as np
from os import walk
import os.path
import re
import time
#from numba import jit, int32, void
from pyLASCA.source import fileNames
from pyLASCA.source import baseLine
from pyLASCA.source import loss
from pyLASCA.source import roiContour

## Analysis Function

In [2]:
#@jit #(void(int32, int32), nopython=True, nogil=False)
def analyze_data_set(effect_id, probandNr, bar):
    effect_alphabet = ("allodynia", "sec_hyperalgesia")
    effect = effect_alphabet[effect_id]
    # Browse Image Filenames
    image_path = "/Users/malkusch/PowerFolders/LaSCA/rawData" 
    image_path += str("/SCTCAPS%s" %(str(probandNr).zfill(2)))
    fn_image = []
    for (dirpath, dirnames, filenames) in walk(image_path):
        fn_image.extend(filenames)
        break
    
    # Browse Coordinate Filenames
    coordinate_main_folder = "/Users/malkusch/PowerFolders/LaSCA/mechanic"
    coordinate_main_folder += str("/SCTCAPS %s" % (str(probandNr).zfill(2)))
    
    # Init Dataframe
    day_array = []
    min_array = []
    frame_array = []
    for i in range (len(fn_image)):
        time_array = fn_image[i].split("_")[1].split(".")
        if (len(time_array) == 1):
            result = re.search(r'\d+', time_array[0])
            if(result):
                day_array.append(int(result.group()))
                min_array.append(-15)
                frame_array.append(int(re.search(r'\d+', fn_image[i].split("_")[2].split(".")[0]).group()))
        if (len(time_array) == 2):
            day_array.append(int(time_array[0]))
            min_array.append(int(time_array[1]))
            frame_array.append(int(re.search(r'\d+', fn_image[i].split("_")[2].split(".")[0]).group()))
    id_array = np.zeros(len(day_array), dtype = int) + probandNr
    effect_array = [effect]*len(day_array)

    df = pd.DataFrame({"Id": id_array,
                      "day": day_array,
                      "time_[min]": min_array,
                       "frame": frame_array,
                       "effect": effect_array
                      })
    
    # Append Pxl Size
    pxl_size_filename = "/Users/malkusch/PowerFolders/LaSCA/Probanden Arm Bilder mit Markierung/pxl_size.xlsx"
    df_pxl = pd.read_excel(io = pxl_size_filename, index_col = 0)
    pxl_size_array = np.zeros(len(df), dtype = float)
    for i in range(len(df)):
        temp_id = int(df["Id"].loc[i])
        temp_day = int(df["day"].loc[i])
        temp_time = int(df["time_[min]"].loc[i])
        pxl_size_array[i] = df_pxl[(df_pxl["object_id"] == temp_id) & (df_pxl["day"] == temp_day) & (df_pxl["time_min"] == temp_time)]["pxlSize_mm"].values[0]

    df["pxlSize_[mm]"] = pxl_size_array
    
    # Append Intensity Statistics
    mean_gray_array = np.zeros(len(df), dtype = float)
    median_gray_array = np.zeros(len(df), dtype = float)
    std_gray_array = np.zeros(len(df), dtype = float)
    min_gray_array = np.zeros(len(df), dtype = int)
    max_gray_array = np.zeros(len(df), dtype = int)
    bl = baseLine.BaseLine()
    for i in range(len(df)):
        if (df["time_[min]"].loc[i] < 0):
            image_filename = image_path + str("/SCTCAPS%s_baseline%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["frame"].loc[i]))
        else:
            image_filename = image_path + str("/SCTCAPS%s_%i.%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i] , df["frame"].loc[i])) 
        if not(os.path.exists(image_filename)):
            raise ValueError('could not find file %s.' % (image_filename))
        image_baseline = cv.imread(image_filename, cv.IMREAD_ANYDEPTH)

        bl.image = image_baseline
        bl.fit()
        mean_gray_array[i] = bl.mean_gray_value
        median_gray_array[i] = bl.median_gray_value
        std_gray_array[i] = bl.std_gray_value
        min_gray_array[i] = bl.min_gray_value
        max_gray_array[i] = bl.max_gray_value


    df["median_[adu]"] = median_gray_array
    df["mean_[adu]"] = mean_gray_array
    df["std_[adu]"] = std_gray_array
    df["min_[adu]"] = min_gray_array
    df["max_[adu]"] = max_gray_array
    
    # Area Alignment
    thr_array = np.zeros(len(df), dtype = float)
    bar.max = len(df)
    bar.value = 1
    for i in range(len(df)):
        bar.value += 1 
        if (df["time_[min]"].loc[i] < 0):
            thr_array[i] = np.nan
            continue
        else:
            image_filename = image_path + str("/SCTCAPS%s_%i.%i_frame%i.tiff" % (str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i] , df["frame"].loc[i])) 
            if not(os.path.exists(image_filename)):
                print('could not find file %s.' % (image_filename))
                thr_array[i] = np.nan
                continue
            coordinate_sub_folder = coordinate_main_folder + str("/Tag %i" %(df["day"].loc[i]))
            coordinate_filename = coordinate_sub_folder + str("/SCTCAPS %s #%i %imin (Colour)_transformed_200612.csv" %(str(df["Id"].loc[i]).zfill(2), df["day"].loc[i], df["time_[min]"].loc[i]))
            if not(os.path.exists(coordinate_filename)):
                print('could not find file %s.' % (coordinate_filename))
                thr_array[i] = np.nan
                continue

            # load data
            img1 = cv.imread(image_filename, cv.IMREAD_ANYDEPTH)
            df_coord = pd.read_csv(coordinate_filename)

            # pre-analysis definition
            df_coord = df_coord[df_coord["effect"] == effect]
            contour_usr = np.zeros([len(df_coord),1,2], dtype = int)
            contour_usr[:,0,0] = np.round(df_coord["x_img_[pxl]"].values).astype(int)
            contour_usr[:,0,1] = np.round(df_coord["y_img_[pxl]"].values).astype(int)

            # run analysis
            roi_usr = roiContour.RoiContour()
            roi_usr.image_raw = img1
            roi_usr.contour = contour_usr
            roi_usr.analyze_roi()

            loss_obj = loss.Loss(obj = roi_usr)
            loss_obj.minimize_brute()

            thr_array[i] = loss_obj.thr_abs


    print("\n")        
    df["thr_[adu]"] = thr_array
    
    # Log Results
    fn = fileNames.FileNames()
    fn.folderName = "/Users/malkusch/PowerFolders/LaSCA/area"
    fn.baseName = str("SCTCAPS%s" %(str(probandNr).zfill(2)))
    fn.supplementalInformation = str("%s" %(effect))
    fn.suffix = "csv"
    fn.updateDateString()
    fn.mergeFileName()
    fn.outFileName

    df.to_csv(path_or_buf = fn.outFileName)
    print("wrote df to %s" %(fn.outFileName))

## User Definitions

In [3]:
proband_array = np.arange(1,20,1)
effect_array = (0,1)

bar_effect = IntProgress(min=0, max=len(effect_array), description='Effect:',bar_style='success')
display(bar_effect) # display the bar

bar_proband = IntProgress(min=0, max=len(proband_array), description='Proband:',bar_style='info')
display(bar_proband) # display the bar

bar_timestep = IntProgress(min=0, max=6, description='Time step:',bar_style='warning')
display(bar_timestep) # display the bar

start = time.time()
for effect_idx in effect_array:
    bar_effect.value += 1
    bar_proband.value = 1
    for probandNr in proband_array:
        bar_proband.value +=1
        analyze_data_set(effect_idx, probandNr, bar_timestep)
end = time.time()
        
print("done: in %is" %(end - start))

IntProgress(value=0, bar_style='success', description='Effect:', max=2)

IntProgress(value=0, bar_style='info', description='Proband:', max=19)

IntProgress(value=0, bar_style='warning', description='Time step:', max=6)

/Users/malkusch/opt/anaconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:1386: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]


Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of

...

Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of

/Users/malkusch/opt/anaconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:1386: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]


Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of

/Users/malkusch/opt/anaconda3/lib/python3.7/site-packages/scipy/ndimage/measurements.py:1386: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]


Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of Loss removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of RoiContour removed form heap
Instance of